In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 假设你的数据是一个Series
#df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements_pycirc.pkl')
df_py = df_good[df_good['buzaki_py_cell_type']=='pyramidal']
df = df_py
base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A"]


for session in sessions:
    # Filter for sessions
    if session == "Total":
        df_a = df
    else:
        df_a = df[df['session'] == session]

    # Separate into control and experimental groups
    control_df = df_a[df_a['animal_id'].isin(control_ids)]
    exp_df = df_a[df_a['animal_id'].isin(exp_ids)]


df_con_deep = control_df[control_df['sub_population']=="deep"].reset_index(drop=True)['Information_content_rate']
df_con_superficial = control_df[control_df['sub_population']=="superficial"].reset_index(drop=True)['Information_content_rate']
df_exp_deep = exp_df[exp_df['sub_population']=="deep"].reset_index(drop=True)['Information_content_rate']
df_exp_superficial = exp_df[exp_df['sub_population']=="superficial"].reset_index(drop=True)['Information_content_rate']

def group_indices_by_step(df_col, step=0.075):
    used_indices = set()
    result = []
    values = df_col.copy()
    max_val = values.max()

    while len(used_indices) < len(values):
        remaining = values[~values.index.isin(used_indices)]
        if remaining.empty:
            break

        current_group = []
        current_val = remaining.min()
        current_idx = remaining.idxmin()
        current_group.append(current_idx)
        used_indices.add(current_idx)

        while True:
            target_val = current_val + step
            remaining = values[~values.index.isin(used_indices)]
            if remaining.empty:
                break

            # 找到大于等于 target_val 的值中最接近 target_val 的那个
            diffs = remaining - target_val
            diffs = diffs[diffs >= 0]
            if diffs.empty:
                break

            next_idx = diffs.idxmin()
            current_val = values[next_idx]
            current_group.append(next_idx)
            used_indices.add(next_idx)

        result.append(current_group)

    return result
results_con_deep = group_indices_by_step(df_con_deep)
results_con_superficial = group_indices_by_step(df_con_superficial)
results_exp_deep = group_indices_by_step(df_exp_deep)
results_exp_superficial = group_indices_by_step(df_exp_superficial)

def plot_rate_map_panel(ax, results_con, df, sublayer="deep", group = "control", map_color='jet'):
    i_row = 0
    base_y = 0.35
    y_step = 0.5

    for indices in results_con:
        y_coord = base_y + (i_row * y_step)

        if len(indices) > 0:
            values = [df[i] for i in indices]
            num_ratemaps = len(indices)
            for i, (idx, value) in enumerate(zip(indices, values)):
                img_path = fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\{group}/{sublayer}/{map_color}/{idx}.png"
                plot_image_at_xy(ax, img_path, value, y_coord, max_size=0.014)
        i_row += 1
    ax.set_xlim(0, 4.5)
    ax.set_ylim(0, 3.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', labelsize=10.5)
    ax.set_xlabel("Information Content Rate (spikes / bit)", fontsize=10.5)
    ax.set_yticks([])
    ax.vlines(x=[1, 2, 3, 4], ymin=0, ymax=8.5, colors='grey', linestyles='dashed', linewidth=3)
    
# Function to load and plot an image with dynamic size adjustment
def plot_image_at_xy(ax, img_path, x, y, max_size=0.09):
    if os.path.exists(img_path):
        img = plt.imread(img_path)
        img_height, img_width = img.shape[:2]
        aspect_ratio = img_width / img_height

        # Calculate zoom based on max_size and aspect ratio
        trans = ax.transData
        fig = ax.get_figure()
        dpi = fig.dpi
        xlim = ax.get_xlim()
        data_width = xlim[1] - xlim[0]
        bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axes_width_pixels = bbox.width * dpi
        pixels_per_data = axes_width_pixels / data_width
        size_pixels = max_size * pixels_per_data
        # zoom = size_pixels / max(img_width, img_height / aspect_ratio)

        zoom = 0.017  # Adjust this value to control ratemap size
        imagebox = OffsetImage(img, zoom=zoom)
        ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0)
        ax.add_artist(ab)
        return ab
    else:
        print(f"Image {img_path} not found")
        return None

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, shapiro, ttest_ind, mannwhitneyu
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np

# Data loading
folder_path = r"S:\Sachuriga\file_with_table\ripple_ch"

def get_pkl_files(folder_path):
    all_files = os.listdir(folder_path)
    pkl_files = [f for f in all_files if f.endswith("withDLC.pkl")]
    return pkl_files

target_prefixes_control = ['65165', '65091', '63383', '66539', '65622']
target_prefixes_exp = ['65588', '63385', '66538', '66537', '66922']
pkl_files = get_pkl_files(folder_path)

# Variables to analyze
variables = ['Information_content_rate', 'Sparsity', 'Field_size', 'Averate_rate', 'bursting_index', 'stability_ma']
titles = ['Information content rate\n (spikes/bit)', 'Sparsity', 'Max Field Size', 'Firing Rate (Hz)', 'Bursting iondex', 'Stability']
all_dfs = []

# Process each pickle file
for file in pkl_files:
    try:
        df = pd.read_pickle(os.path.join(folder_path, file))
    except Exception as e:
        continue
    #df = df[(df['cell_type'] == "pyramidal") & (df['session'] == "A")]

    df = df[(df['buzaki_py_cell_type'] == "pyramidal") & (df['session'] == "A")]
    #df = df[df['session'] == "A"]
    if df.empty:
        print(f"Warning: File {file} has no pyramidal cells")
        continue
    try:
        animal_id = df['animal_id'].iloc[0]
    except KeyError:
        print(f"Warning: File {file} does not have 'animal_id' column")
        continue
    if any(animal_id.startswith(prefix) for prefix in target_prefixes_control):
        df['group_ani'] = 'control'
    elif any(animal_id.startswith(prefix) for prefix in target_prefixes_exp):
        df['group_ani'] = 'exp'
    else:
        print(f"Warning: animal_id {animal_id} does not match any group")
        continue
    all_dfs.append(df)

y_pos = "addjust y r2"
combined_df=[]
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['depth'] = combined_df[y_pos].apply(lambda x: 'deep' if x > 0 else 'superficial')
combined_df['group_depth'] = combined_df['group_ani'] + '_' + combined_df['depth']

# Convert variables to numeric and handle invalid values
for var in variables:
    combined_df[var] = pd.to_numeric(combined_df[var], errors='coerce')
    if combined_df[var].isna().any():
        print(f"Warning: {var} contains NaN values after conversion to numeric")

# Function to test normality and choose test
def choose_stat_test(data1, data2, var_name, group1_name, group2_name):
    # Drop NaN values and ensure numeric
    data1 = data1.dropna()
    data2 = data2.dropna()
    
    # Check for non-numeric values
    if not np.issubdtype(data1.dtype, np.number) or not np.issubdtype(data2.dtype, np.number):
        print(f"Error: Non-numeric data detected in {var_name} for {group1_name} or {group2_name}")
        print(f"{group1_name} dtype: {data1.dtype}, {group2_name} dtype: {data2.dtype}")
        print(f"{group1_name} sample: {data1.head()}")
        print(f"{group2_name} sample: {data2.head()}")
        return "Invalid", np.nan, np.nan

    # Check if data is empty after dropping NaNs
    if len(data1) == 0 or len(data2) == 0:
        print(f"Error: Empty dataset for {var_name} in {group1_name} or {group2_name} after dropping NaNs")
        return "Empty", np.nan, np.nan

    # Perform Shapiro-Wilk test for normality
    stat1, p1 = shapiro(data1)
    stat2, p2 = shapiro(data2)
    
    print(f"{var_name} - {group1_name} Shapiro-Wilk: p={p1:.4f}")
    print(f"{var_name} - {group2_name} Shapiro-Wilk: p={p2:.4f}")
    
    # Choose test based on normality
    if p1 > 0.05 and p2 > 0.05:  # Both are normal
        print(f"{var_name} - Using t-test (both groups normal)")
        stat, p = ttest_ind(data1, data2, equal_var=True)  # Assuming equal variances
        test_name = "t-test"
    else:
        print(f"{var_name} - Using Mann-Whitney U test (non-normal distribution)")
        stat, p = mannwhitneyu(data1, data2, alternative='two-sided')
        test_name = "Mann-Whitney U"
    
    return test_name, stat, p

group_ani = []
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
for index, row in combined_df.iterrows():
    if str(row['matlab_animal']) in control_ids:
        group_ani.append("control")
    else:
        group_ani.append("exp")
combined_df['group_ani']=None
combined_df['group_ani']=group_ani


In [ ]:
from scipy.stats import binomtest
control_df_deep = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['sub_population'] == 'deep')]
exp_df_deep = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['sub_population'] == 'deep')]

# # Control_superficial vs. Exp_superficial
# control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'superficial')][var]
# exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'superficial')][var]

control_df_place = control_df_deep[(control_df_deep['h_0_place_cell'] == 1) & 
                                (control_df_deep['Information_content_rate'] >= 1.68) & 
                                (control_df_deep['matlab_maxfsize'] >= 20) ]
exp_df_place = exp_df_deep[(exp_df_deep['h_0_place_cell'] == 1) & 
                        (exp_df_deep['Information_content_rate'] >= 1.68) & 
                        (exp_df_deep['matlab_maxfsize'] >= 20) ]
# Calculate numbers for pie charts
control_place_deep = len(control_df_place)
control_non_place_deep = len(control_df_deep) - control_place_deep
exp_place_deep = len(exp_df_place)
exp_non_place_deep = len(exp_df_deep) - exp_place_deep
result = binomtest(exp_place_deep, len(exp_df_deep), p=(control_place_deep /len(control_df_deep)), alternative='less')
p_value_deep = result.pvalue
p_value_deep

In [ ]:
from scipy.stats import binomtest
control_df_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['sub_population'] == 'superficial')]
exp_df_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['sub_population'] == 'superficial')]

# # Control_superficial vs. Exp_superficial
# control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'superficial')][var]
# exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'superficial')][var]

control_df_place_superficial = control_df_superficial[(control_df_superficial['h_0_place_cell'] == 1) & 
                                (control_df_superficial['Information_content_rate'] >= 1.68) & 
                                (control_df_superficial['matlab_maxfsize'] >= 20)]
exp_df_place_superficial = exp_df_superficial[(exp_df_superficial['h_0_place_cell'] == 1) & 
                        (exp_df_superficial['Information_content_rate'] >= 1.68) & 
                        (exp_df_superficial['matlab_maxfsize'] >= 20) ]

# Calculate numbers for pie charts
control_place_superficial = len(control_df_place_superficial)
control_non_place_superficial = len(control_df_superficial) - control_place_superficial
exp_place_superficial = len(exp_df_place_superficial)
exp_non_place_superficial = len(exp_df_superficial) - exp_place_superficial
result = binomtest(exp_place_superficial, len(exp_df_superficial), p=(control_place_superficial /len(control_df_superficial)), alternative='less')
p_value_superficial = result.pvalue
p_value_superficial

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.collections import LineCollection

fig = plt.figure(figsize=(7.2, 11), dpi=1200)
plt.rcParams.update({'font.size': 8})
plt.rcParams['font.family'] = 'Times New Roman'


gs = gridspec.GridSpec(7, 8, height_ratios=[1.5,1,1,1,1,1,1], width_ratios=[0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])  # First row taller
plt.rcParams.update({
    'axes.labelpad': -0.1,
    'ytick.major.pad': -0.1,
    'xtick.major.pad': -0.1,
    'ytick.major.size': 2,
    'xtick.major.size': 2
})

linewidth = 0.5
ax1_1 = fig.add_subplot(gs[0, 0])
ax1_1.axis('off')


ax1_2 = fig.add_subplot(gs[0, 1])
import numpy as np
import matplotlib.pyplot as plt

# Parameters for the simulation
fs = 1000  # Sampling frequency (Hz)
t = np.linspace(0, 0.5, int(fs * 1))  # Time vector (0 to 0.5 seconds)
n_channels = 11  # Number of channels (rows)
f_ripple = 150  # Frequency of the ripple (150 Hz, typical for hippocampal SWR)
noise_level = 0.1  # Small noise to make it realistic

# Amplitudes
amplitudes = (np.float32([0.15066703, 0.15041492, 0.15354033, 0.15755153, 0.16833866, 0.17786268, 0.18966555, 0.19084985, 0.18718866, 0.18549542, 0.17947777])*10)**5
amplitudes = amplitudes / np.max(amplitudes) / 0.5

# Centers for the ripples
centers = [0.23, 0.27]  # Alternating centers

# Generate ripple signals for each channel
signals = []
for i in range(n_channels):
    signal = np.zeros_like(t)
    center = centers[i % 2]
    amp = amplitudes[i]
    envelope = amp * np.exp(-((t - center) ** 2) / (2 * 0.01 ** 2))
    ripple = envelope * np.sin(2 * np.pi * f_ripple * t)
    noise = noise_level * np.random.randn(len(t))
    signal += ripple + noise
    signals.append(signal)

# Find the channel with the largest amplitude
max_amp_idx = 6

# Plot the signals

for i in range(n_channels):
    plot_idx = i  # Keep original index for signals
    if plot_idx == max_amp_idx:
        ax1_2.plot(t, signals[plot_idx] + i * 2, color='red', linewidth=linewidth, label='Max Amplitude')
    else:
        ax1_2.plot(t, signals[plot_idx] + i * 2, color='black', linewidth=linewidth)

ax1_2.invert_yaxis()  # Reverse the y-axis
ax1_2.axis('off')  # Remove axes
ax1_2.set_xlim(.2,.3)


ax1_3 = fig.add_subplot(gs[0, 2:6])
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from probeinterface import Probe, ProbeGroup
from probeinterface import get_probe
import probeinterface as pi
from probeinterface.plotting import plot_probe, plot_probe_group


# Load probe
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-236-F'
probe = pi.get_probe(manufacturer, probe_name)
print(probe)
df= pd.read_pickle(r"S:\Sachuriga\file_with_table\ripple_ch/63383_2024-07-25_A_units_table_withDLC.pkl")
#df_py = df_list_actual
#df = df[df['buzaki_py_cell_type']=="pyramidal"]
# Map channels to device indices
mapping_to_device = [
    41, 39, 38, 37, 35, 34, 33, 32, 29, 30, 28, 26, 25, 24, 22, 20,
    46, 45, 44, 43, 42, 40, 36, 31, 27, 23, 21, 18, 19, 17, 16, 14,
    55, 53, 54, 52, 51, 50, 49, 48, 47, 15, 13, 12, 11, 9, 10, 8,
    63, 62, 61, 60, 59, 58, 57, 56, 7, 6, 5, 4, 3, 2, 1, 0
]
probe.set_device_channel_indices(mapping_to_device)

# Create probe dataframe
probe_df = probe.to_dataframe(complete=True)
print(probe_df.loc[:, ["contact_ids", "shank_ids", "device_channel_indices"]])

# Create probegroup and plot probe
probegroup = ProbeGroup()
probegroup.add_probe(probe)
plot_probe(probe, contacts_colors="grey",ax=ax1_3)

# Assuming df is your dataframe with x, y coordinates
df_py = df[df['addjust y r2']>0]  # Replace with your actual dataframe
x = df_py['x']
y = df_py['y']
# Create scatter plot with triangle markers
# First scatter plot (blue triangles, edge only)
sns.scatterplot(x=x, y=y, s=50, alpha=1, marker='^', edgecolor='orange', facecolor='none',ax=ax1_3)

# Assuming df is your dataframe with x, y coordinates
df_py = df[df['addjust y r2']<=0]  # Replace with your actual dataframe
x = df_py['x']
y = df_py['y']
# Second scatter plot (black triangles, edge only)
sns.scatterplot(x=x, y=y, s=50, alpha=1, marker='^', edgecolor='darkviolet', facecolor='none',ax=ax1_3)
# Plot short horizontal lines for specified channels
chs = [20, 6, 28, 53, 44]
line_length = 100  # Adjust the length of the horizontal line as needed

for ch in chs:
    temp = probe_df[probe_df['device_channel_indices'] == ch]
    if not temp.empty:
        x, y = temp['x'].iloc[0], temp['y'].iloc[0]
        # Draw a short horizontal line centered at (x, y)
        ax1_3.plot([x - line_length/2, x + line_length/2], [y, y], 'k--', linewidth=1)

plt.axis('off')  # Hides the entire axis (including ticks, labels, and spines)
plt.title('')    # Ensures no title is displayed

ax1_4 = fig.add_subplot(gs[0, 6:8])

y_pos = "addjust y r2"
#combined_df=pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/4depth.pkl")
# Create barplot with adjusted width to reduce distance
sns.barplot(data=combined_df, x='group_ani', y=y_pos, hue='group_depth', ax=ax1_4, width=1)
ax1_4.patch.set_linewidth(0.1)
# Customize bars for superficial layers (no face color, only edge)
for patch in ax1_4.patches:
    
    hue = ax1_4.get_legend().get_texts()[ax1_4.patches.index(patch) % len(ax1_4.get_legend().get_texts())].get_text()
    if 'superficial' in hue:
        patch.set_facecolor('#B755E1')  # Remove face color
        if 'control' in hue:
            patch.set_edgecolor('blue')  # Blue edge for control superficial
        elif 'exp' in hue:
            patch.set_edgecolor('red')  # Cyan edge for exp superficial
    else:
        patch.set_facecolor('#FFBB41')
        if 'control' in hue:
            patch.set_edgecolor('blue')  # Blue edge for control deep
        elif 'exp' in hue:
            patch.set_edgecolor('red')  # Cyan edge for exp deep

#ax3.set_title('Mean soma position by group and depth')
ax1_4.set_xlabel([]).set_visible(False)
ax1_4.set_ylabel('μm')
ax1_4.spines['top'].set_visible(False)
ax1_4.spines['right'].set_visible(False)
ax1_4.get_legend().remove()  # Remove the legend as requested
ax1_4.set_yticks([-20,0,20])
#ax3.set_xticks = ([0.2,0.8])
ax1_4.set_xticklabels(["CRs +","CRs -"],rotation=-30)
ax1_4.set_xlim(-1,2)

ax2 = fig.add_subplot(gs[1, :])
ax3 = fig.add_subplot(gs[2, :])

ax5 = fig.add_subplot(gs[4, :])
ax6 = fig.add_subplot(gs[5, :])


plot_rate_map_panel(ax2, results_con_deep, df_con_deep,sublayer="deep")
plot_rate_map_panel(ax3, results_exp_deep, df_exp_deep,sublayer="deep", group = "exp")

plot_rate_map_panel(ax5, results_con_superficial, df_con_superficial,sublayer="superficial")
plot_rate_map_panel(ax6, results_exp_superficial, df_exp_superficial,sublayer="superficial", group = "exp")

## funtional properties
ax4_1 = fig.add_subplot(gs[3, 0])
ax4_2 = fig.add_subplot(gs[3, 1])
ax4_3 = fig.add_subplot(gs[3, 2])
ax4_4 = fig.add_subplot(gs[3, 3])
ax4_5 = fig.add_subplot(gs[3, 4])
ax4_6 = fig.add_subplot(gs[3, 5])
ax4_7 = fig.add_subplot(gs[3, 6:8])



axis = [ax4_1,ax4_2,ax4_3,ax4_4,ax4_5,ax4_6,ax4_7]

palette = {'control': 'blue', 'exp': 'red'}
#combined_df=pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/functional_properties_with_python_measurements.pkl")
for i, (var, title) in enumerate(zip(variables, titles)):
    # Control_deep vs. Exp_deep
    control_deep = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['sub_population'] == 'deep')][var]
    exp_deep = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['sub_population'] == 'deep')][var]
    n_deep=len(control_deep)
    en_deep=len(exp_deep)
    print(fr"n numbers con deep {n_deep} n number exp deep {en_deep}")
    # Choose test for deep
    test_name_deep, stat_deep, p_valued = choose_stat_test(control_deep, exp_deep, title, "Control_deep", "Exp_deep")
    print(f"{title} - Control_deep vs. Exp_deep ({test_name_deep}): statistic={stat_deep:.3f}, p-value={p_valued:.4f}")
    
    # Control_superficial vs. Exp_superficial
    control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['sub_population'] == 'superficial')][var]
    exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['sub_population'] == 'superficial')][var]
    
    # Choose test for superficial
    test_name_sup, stat_sup, p_values = choose_stat_test(control_superficial, exp_superficial, title, "Control_superficial", "Exp_superficial")
    print(f"{title} - Control_superficial vs. Exp_superficial ({test_name_sup}): statistic={stat_sup:.3f}, p-value={p_values:.4f}")
    ks_stat_deep, p_value_d = ks_2samp(control_deep, exp_deep, nan_policy='omit')
    ks_stat_sup, p_values_s = ks_2samp(control_superficial, exp_superficial, nan_policy='omit')

    # # Deep subplot
    # sns.violinplot(
    #     data=combined_df[combined_df['depth'] == 'deep'],x='group_ani', y=var, ax=axis[i], hue='group_ani',inner = "quartiles",
    #     palette={"control": "blue", "exp": "red"}, cut=0, edgecolor='white'
    # )
    # sns.swarmplot(
    #             data=combined_df[combined_df['depth'] == 'deep'],x='group_ani', y=var, ax=axis[i], size=.5,
    #             hue='group_ani', palette={"control": "black", "exp": "black"},
    #             alpha=1, legend=False
    #         )
    
    sns.violinplot(
        data=combined_df[combined_df['depth'] == 'deep'], 
        x='group_ani', 
        y=var, 
        ax=axis[i], 
        hue='group_ani', 
        inner=None,
        palette={"control": "blue", "exp": "red"},
        cut=0, 
        linewidth=0  # Removes the outline
    )

    # Boxplot: only edge lines with midline (median), hide outliers, no fill, no mean marker
    # Adjusted to make the boxplot lines narrower (thinner) by setting linewidth=1 (adjust as needed, e.g., 0.5 for even thinner)
    sns.boxplot(
        data=combined_df[combined_df['depth'] == 'deep'], 
        x='group_ani', 
        y=var, 
        ax=axis[i], 
        hue='group_ani', 
        palette={"control": "black", "exp": "black"},
        width=0.5, 
        fill=False,  # No fill, only outlines
        showfliers=False,  # Hide outliers
        showmeans=False,  # Remove mean marker, assuming midline is the median
        linewidth=0.5,  # Makes the lines narrower (thinner)
        boxprops={"zorder": 3}
    )
    for l in axis[i].get_children():
        if isinstance(l, LineCollection):  # Quartile lines are LineCollections
            l.set_linewidth(0.5)
    axis[i].set_xlabel(title).set_visible(False)
    axis[i].set_ylabel(f'{title}')
    axis[i].legend().set_visible(False)
    axis[i].spines['top'].set_visible(False)
    axis[i].spines['right'].set_visible(False)
    ax=axis[i]
    ax.set_xticklabels(['CRs +', 'CRs -'], rotation = -30)
    y_max = ax.get_ylim()[1]
    bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
    x_positions = [0, 1]  # Adjusted positions for 'Control' and 'Experimental' groups
    p_val = p_valued
    if (p_val < 0.05) & (p_val > 0.01):
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')
    elif (p_val < 0.01) & (p_val > 0.001):
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'**', ha='center', va='bottom')
    elif p_val < 0.001:
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'***', ha='center', va='bottom')



# Data
groups = ['CRs +', 'CRs -']
counts = [
    [round(control_place_deep/len(control_df_deep),2)*100, round(control_non_place_deep/len(control_df_deep),2)*100], 
    [round(exp_place_deep/len(exp_df_deep),2)*100, round(exp_non_place_deep/len(exp_df_deep),2)*100], 
]
percentages = [
    [round(control_place_deep/len(control_df_deep),2)*100, round(control_non_place_deep/len(control_df_deep),2)*100], 
    [round(exp_place_deep/len(exp_df_deep),2)*100, round(exp_non_place_deep/len(exp_df_deep),2)*100], 
]

# Colors
colors = ['blue', 'lightblue']  # For CRs +
colors1 = ['red', '#FF6666']    # For CRs -

# Create bar chart
ax=ax4_7

# Set positions for bars to be closer
bar_width = 0.35  # Reduced width to bring bars closer
x = np.arange(len(groups))  # Create positions for the bars
x = x * 0.5  # Scale x positions to reduce gap between bars

# Plot stacked bars
bars = []
for i in range(len(counts[0])):  # For each category (0 and 1)
    # Create bars for each group with appropriate colors
    bar = ax.bar(x, [counts[j][i] for j in range(len(counts))], 
                 bottom=[sum(counts[j][:i]) for j in range(len(counts))], 
                 color=[colors[i] if j == 0 else colors1[i] for j in range(len(counts))], 
                 label=f'Category {i+1}' if i == 0 else None, 
                 width=bar_width)
    bars.append(bar)

# Add count and percentage labels on top of each segment
for i, bar_group in enumerate(bars):
    if i==0:
        cell_type = "(Place cell)"
    else:
        cell_type = "(None \nplace cell)"
    for j, bar in enumerate(bar_group):
        height = bar.get_height()
        total = sum(counts[j])
        bottom = sum(counts[j][:i])
        ax.text(bar.get_x() + bar.get_width()/2, bottom + height/2, 
                f'{percentages[j][i]}%\n {cell_type}', ha='center', va='center', rotation=0, color='white',fontsize=5)

# Customize the plot
ax.set_ylabel('%Neurons')
ax.set_xticks(x)
ax.set_xticklabels(groups, rotation = -30)
ax.legend().set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
p_val = p_value_deep
y_max = ax.get_ylim()[1]
bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
x_positions = x  # Adjusted positions for 'Control' and 'Experimental' groups
if (p_val < 0.05) & (p_val > 0.01):
    ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                        color='black', lw=1.5)
    ax.text(x[1]*0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')


ax7_1 = fig.add_subplot(gs[6, 0])
ax7_2 = fig.add_subplot(gs[6, 1])
ax7_3 = fig.add_subplot(gs[6, 2])
ax7_4 = fig.add_subplot(gs[6, 3])
ax7_5 = fig.add_subplot(gs[6, 4])
ax7_6 = fig.add_subplot(gs[6, 5])
ax7_7 = fig.add_subplot(gs[6, 6:8])


axis = [ax7_1,ax7_2,ax7_3,ax7_4,ax7_5,ax7_6]
for i, (var, title) in enumerate(zip(variables, titles)):
    # # Control_deep vs. Exp_deep
    # control_deep = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'deep')][var]
    # exp_deep = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'deep')][var]
    
    # # Choose test for deep
    # test_name_deep, stat_deep, p_valued = choose_stat_test(control_deep, exp_deep, title, "Control_deep", "Exp_deep")
    # print(f"{title} - Control_deep vs. Exp_deep ({test_name_deep}): statistic={stat_deep:.3f}, p-value={p_valued:.4f}")
    
    # Control_superficial vs. Exp_superficial
    control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['sub_population'] == 'superficial')][var]
    exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['sub_population'] == 'superficial')][var]
    
    # Choose test for superficial
    test_name_sup, stat_sup, p_values = choose_stat_test(control_superficial, exp_superficial, title, "Control_superficial", "Exp_superficial")
    print(f"{title} - Control_superficial vs. Exp_superficial ({test_name_sup}): statistic={stat_sup:.3f}, p-value={p_values:.4f}")
    ks_stat_deep, p_value_d = ks_2samp(control_deep, exp_deep, nan_policy='omit')
    ks_stat_sup, p_values_s = ks_2samp(control_superficial, exp_superficial, nan_policy='omit')

    # # Deep subplot
    # sns.violinplot(
    #     data=combined_df[combined_df['depth'] == 'superficial'],x='group_ani', y=var, ax=axis[i], hue='group_ani',inner = "quartiles",
    #     palette={"control": "blue", "exp": "red"}, cut=0, edgecolor=None
    # )
    sns.violinplot(
        data=combined_df[combined_df['depth'] == 'superficial'], 
        x='group_ani', 
        y=var, 
        ax=axis[i], 
        hue='group_ani', 
        inner=None,
        palette={"control": "blue", "exp": "red"},
        cut=0, 
        linewidth=0  # Removes the outline
    )

    # Boxplot: only edge lines with midline (median), hide outliers, no fill, no mean marker
    # Adjusted to make the boxplot lines narrower (thinner) by setting linewidth=1 (adjust as needed, e.g., 0.5 for even thinner)
    sns.boxplot(
        data=combined_df[combined_df['depth'] == 'superficial'], 
        x='group_ani', 
        y=var, 
        ax=axis[i], 
        hue='group_ani', 
        palette={"control": "black", "exp": "black"},
        width=0.5, 
        fill=False,  # No fill, only outlines
        showfliers=False,  # Hide outliers
        showmeans=False,  # Remove mean marker, assuming midline is the median
        linewidth=0.5,  # Makes the lines narrower (thinner)
        boxprops={"zorder": 3}
    )
    # sns.swarmplot(
    #             data=combined_df[combined_df['depth'] == 'superficial'],x='group_ani', y=var, ax=axis[i], size=.5,
    #             hue='group_ani', palette={"control": "black", "exp": "black"},
    #             alpha=1, legend=False
    #         )
    
    axis[i].set_xlabel(title).set_visible(False)
    for l in axis[i].get_children():
        if isinstance(l, LineCollection):  # Quartile lines are LineCollections
            l.set_linewidth(0.5)
    axis[i].set_ylabel(f'{title}')
    axis[i].legend().set_visible(False)
    axis[i].spines['top'].set_visible(False)
    axis[i].spines['right'].set_visible(False)
    ax=axis[i]
    ax.set_xticklabels(['CRs +', 'CRs -'], rotation = -30)
    y_max = ax.get_ylim()[1]
    bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
    x_positions = [0, 1]  # Adjusted positions for 'Control' and 'Experimental' groups
    p_val = p_values
    if (p_val < 0.05) & (p_val > 0.01):
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')
    elif (p_val < 0.01) & (p_val > 0.001):
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'**', ha='center', va='bottom')
    elif p_val < 0.001:
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'***', ha='center', va='bottom')

# Create bar chart
ax=ax7_7

# Data
groups = ['CRs +', 'CRs -']
counts = [
    [round(control_place_superficial/len(control_df_superficial),2)*100, round(control_non_place_superficial/len(control_df_superficial),2)*100], 
    [round(exp_place_superficial/len(exp_df_superficial),2)*100, round(exp_non_place_superficial/len(exp_df_superficial),2)*100], 
]
percentages = [
    [round(control_place_superficial/len(control_df_superficial),2)*100, round(control_non_place_superficial/len(control_df_superficial),2)*100], 
    [round(exp_place_superficial/len(exp_df_superficial),2)*100, round(exp_non_place_superficial/len(exp_df_superficial),2)*100], 
]

# Colors
colors = ['blue', 'lightblue']  # For CRs +
colors1 = ['red', '#FF6666']    # For CRs -

# Create bar chart
ax=ax7_7
# Set positions for bars to be closer
bar_width = 0.35  # Reduced width to bring bars closer
x = np.arange(len(groups))  # Create positions for the bars
x = x * 0.5  # Scale x positions to reduce gap between bars

# Plot stacked bars
bars = []
for i in range(len(counts[0])):  # For each category (0 and 1)
    # Create bars for each group with appropriate colors
    bar = ax.bar(x, [counts[j][i] for j in range(len(counts))], 
                 bottom=[sum(counts[j][:i]) for j in range(len(counts))], 
                 color=[colors[i] if j == 0 else colors1[i] for j in range(len(counts))], 
                 label=f'Category {i+1}' if i == 0 else None, 
                 width=bar_width)
    bars.append(bar)

# Add count and percentage labels on top of each segment
for i, bar_group in enumerate(bars):
    if i==0:
        cell_type = "(Place cell)"
    else:
        cell_type = "(None \nplace cell)"
    for j, bar in enumerate(bar_group):
        height = bar.get_height()
        total = sum(counts[j])
        bottom = sum(counts[j][:i])
        ax.text(bar.get_x() + bar.get_width()/2, bottom + height/2, 
                f'{percentages[j][i]}%\n {cell_type}', ha='center', va='center', rotation=0, color='white',fontsize=5)

# Customize the plot
ax.set_ylabel('%Neurons')
ax.set_xticks(x)
ax.set_xticklabels(groups, rotation = -30)
ax.legend().set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
p_val = p_value_superficial
y_max = ax.get_ylim()[1]
bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
x_positions = x  # Adjusted positions for 'Control' and 'Experimental' groups
if (p_val < 0.05) & (p_val > 0.01):
    ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                        color='black', lw=1.5)
    ax.text(x[1]*0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')

plt.rcParams.update({'font.size': 7})
plt.tight_layout()
fig.subplots_adjust(top=0.92, bottom=0.08, left=0.1, right=0.95, hspace=1.1, wspace=1.3)
plt.savefig(r'Q:\sachuriga\CR_CA1_paper\Figures/fig5.png', transparent=True, dpi=1200, bbox_inches='tight')
plt.show()


In [ ]:
fig = plt.figure(figsize=(7.2, 11), dpi=1200)
plt.rcParams.update({'font.size': 8})
gs = gridspec.GridSpec(7, 8, height_ratios=[1.5,1,1,1,1,1,1], width_ratios=[0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])  # First row taller
plt.rcParams.update({
    'axes.labelpad': -0.1,
    'ytick.major.pad': -0.1,
    'xtick.major.pad': -0.1,
    'ytick.major.size': 2,
    'xtick.major.size': 2
})

linewidth = 0.5
ax1_1 = fig.add_subplot(gs[0, 0])
ax1_1.axis('off')
ax7_1 = fig.add_subplot(gs[6, 0])
ax7_2 = fig.add_subplot(gs[6, 1])
ax7_3 = fig.add_subplot(gs[6, 2])
ax7_4 = fig.add_subplot(gs[6, 3])
ax7_5 = fig.add_subplot(gs[6, 4])
ax7_6 = fig.add_subplot(gs[6, 5])
ax7_7 = fig.add_subplot(gs[6, 6:8])


axis = [ax7_1,ax7_2,ax7_3,ax7_4,ax7_5,ax7_6]
for i, (var, title) in enumerate(zip(variables, titles)):
    # # Control_deep vs. Exp_deep
    # control_deep = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'deep')][var]
    # exp_deep = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'deep')][var]
    
    # # Choose test for deep
    # test_name_deep, stat_deep, p_valued = choose_stat_test(control_deep, exp_deep, title, "Control_deep", "Exp_deep")
    # print(f"{title} - Control_deep vs. Exp_deep ({test_name_deep}): statistic={stat_deep:.3f}, p-value={p_valued:.4f}")
    
    # Control_superficial vs. Exp_superficial
    control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'superficial')][var]
    exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'superficial')][var]
    
    # Choose test for superficial
    test_name_sup, stat_sup, p_values = choose_stat_test(control_superficial, exp_superficial, title, "Control_superficial", "Exp_superficial")
    print(f"{title} - Control_superficial vs. Exp_superficial ({test_name_sup}): statistic={stat_sup:.3f}, p-value={p_values:.4f}")
    ks_stat_deep, p_value_d = ks_2samp(control_deep, exp_deep, nan_policy='omit')
    ks_stat_sup, p_values_s = ks_2samp(control_superficial, exp_superficial, nan_policy='omit')

    # Deep subplot
    # sns.violinplot(
    #     data=combined_df[combined_df['depth'] == 'superficial'],x='group_ani', y=var, ax=axis[i], hue='group_ani',inner = "quartiles",
    #     palette={"control": "blue", "exp": "red"}, cut=0, edgecolor=None
    # )
    # Violin plot: only shaded area (filled with color, no outline)

    sns.violinplot(
        data=combined_df[combined_df['depth'] == 'superficial'], 
        x='group_ani', 
        y=var, 
        ax=axis[i], 
        hue='group_ani', 
        inner=None,
        palette={"control": "blue", "exp": "red"},
        cut=0, 
        linewidth=0  # Removes the outline
    )

    # Boxplot: only edge lines with midline (median), hide outliers, no fill, no mean marker
    # Adjusted to make the boxplot lines narrower (thinner) by setting linewidth=1 (adjust as needed, e.g., 0.5 for even thinner)
    sns.boxplot(
        data=combined_df[combined_df['depth'] == 'superficial'], 
        x='group_ani', 
        y=var, 
        ax=axis[i], 
        hue='group_ani', 
        palette={"control": "black", "exp": "black"},
        width=0.5, 
        fill=False,  # No fill, only outlines
        showfliers=False,  # Hide outliers
        showmeans=False,  # Remove mean marker, assuming midline is the median
        linewidth=0.5,  # Makes the lines narrower (thinner)
        boxprops={"zorder": 3}
    )
    axis[i].set_xlabel(title).set_visible(False)
    axis[i].set_ylabel(f'{title}')
    axis[i].legend().set_visible(False)
    axis[i].spines['top'].set_visible(False)
    axis[i].spines['right'].set_visible(False)
    ax=axis[i]
    ax.set_xticklabels(['CRs +', 'CRs -'], rotation = -30)
    y_max = ax.get_ylim()[1]
    bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
    x_positions = [0, 1]  # Adjusted positions for 'Control' and 'Experimental' groups
    p_val = p_values
    if (p_val < 0.05) & (p_val > 0.01):
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')
    elif (p_val < 0.01) & (p_val > 0.001):
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'**', ha='center', va='bottom')
    elif p_val < 0.001:
        ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        ax.text(0.5, y_max + bar_height * 1.1, f'***', ha='center', va='bottom')

# Create bar chart
ax=ax7_7

# Data
groups = ['CRs +', 'CRs -']
counts = [
    [round(control_place_superficial/len(control_df_superficial),2)*100, round(control_non_place_superficial/len(control_df_superficial),2)*100], 
    [round(exp_place_superficial/len(exp_df_superficial),2)*100, round(exp_non_place_superficial/len(exp_df_superficial),2)*100], 
]
percentages = [
    [round(control_place_superficial/len(control_df_superficial),2)*100, round(control_non_place_superficial/len(control_df_superficial),2)*100], 
    [round(exp_place_superficial/len(exp_df_superficial),2)*100, round(exp_non_place_superficial/len(exp_df_superficial),2)*100], 
]

# Colors
colors = ['blue', 'lightblue']  # For CRs +
colors1 = ['red', '#FF6666']    # For CRs -

# Create bar chart
ax=ax7_7
# Set positions for bars to be closer
bar_width = 0.35  # Reduced width to bring bars closer
x = np.arange(len(groups))  # Create positions for the bars
x = x * 0.5  # Scale x positions to reduce gap between bars

# Plot stacked bars
bars = []
for i in range(len(counts[0])):  # For each category (0 and 1)
    # Create bars for each group with appropriate colors
    bar = ax.bar(x, [counts[j][i] for j in range(len(counts))], 
                 bottom=[sum(counts[j][:i]) for j in range(len(counts))], 
                 color=[colors[i] if j == 0 else colors1[i] for j in range(len(counts))], 
                 label=f'Category {i+1}' if i == 0 else None, 
                 width=bar_width)
    bars.append(bar)

# Add count and percentage labels on top of each segment
for i, bar_group in enumerate(bars):
    if i==0:
        cell_type = "(Place cell)"
    else:
        cell_type = "(None \nplace cell)"
    for j, bar in enumerate(bar_group):
        height = bar.get_height()
        total = sum(counts[j])
        bottom = sum(counts[j][:i])
        ax.text(bar.get_x() + bar.get_width()/2, bottom + height/2, 
                f'{percentages[j][i]}%\n {cell_type}', ha='center', va='center', rotation=0, color='white',fontsize=5)

# Customize the plot
ax.set_ylabel('%Neurons')
ax.set_xticks(x)
ax.set_xticklabels(groups, rotation = -30)
ax.legend().set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
p_val = p_value_superficial
y_max = ax.get_ylim()[1]
bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
x_positions = x  # Adjusted positions for 'Control' and 'Experimental' groups
if (p_val < 0.05) & (p_val > 0.01):
    ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                        color='black', lw=1.5)
    ax.text(x[1]*0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')


plt.rcParams.update({'font.size': 7})
plt.tight_layout()
fig.subplots_adjust(top=0.92, bottom=0.08, left=0.1, right=0.95, hspace=1.1, wspace=1.3)
#plt.savefig(r'Q:\sachuriga\CR_CA1_paper\Figures/fig5.png', transparent=True, dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
counts

In [ ]:
control_non_place_deep

In [ ]:
group_ani = []
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
for index, row in combined_df.iterrows():
    if str(row['matlab_animal']) in control_ids:
        group_ani.append("control")
    else:
        group_ani.append("exp")
combined_df['group_ani']=None
combined_df['group_ani']=group_ani

In [ ]:
combined_df.to_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/functional_properties_with_python_measurements.pkl")